<b>IMPORT</b> </br>
Importing the required libraries, including tensorflow, keras, pandas, numpy and flower

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
from tensorflow.keras import Model
from tensorflow.keras.metrics import BinaryAccuracy, AUC, Precision, Recall
from tensorflow.keras.applications import DenseNet121, VGG16, ResNet50
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.activations import sigmoid, softmax
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import preprocessing
from tensorflow.keras import backend
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import random
import math
import tensorflow_addons as tfa
import flwr as fl
from lime import lime_image
from lime.wrappers.scikit_image import SegmentationAlgorithm
from skimage.segmentation import mark_boundaries

<b>ENVIRONMENT SETTINGS</b></br>
Disabling tensorflow warnings and allowing GPU growth

In [ ]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3" #disabling tensorflow warnings
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true" #allowing GPU growth


<b>CLIENT SETTINGS</b></br>
Defining the number of total clients and this clients ID

In [ ]:
clients = 5
client_num = 5

<b>STANDARD VALUES</b></br>
Setting the standard values for model training such as image size, batch size, epochs, validation split, etc.

In [ ]:
#images are entered into the neural network with a resolution of IMAGE_SIZE x IMAGE_SIZE
IMAGE_SIZE = 320

#three channels for the pixel representation of a color image
CHANNELS = 3 

#batch size, epochs and learning rate
BATCH_SIZE = 8
EPOCHS = 3
LEARNING_RATE = 0.001

#number of images from the dataset used for training & validation (maximum: 191027)
IMAGES = 191027

#80 % of the selected images are used for training, 20% for validation
TRAIN_VALIDATION_SPLIT = 0.8

#image agmentation is used with 50% of training images being augmented
IMAGE_AUGMENTATION = True
AUGMENTATION_SPLIT = 0.5

#sigmoid activatino function is used in the final dense layer for classification
ACTIVATION =  sigmoid

#weights are initialised in the first epoch using a uniform distribution
INITIALIZER = GlorotUniform(seed = 42)

#the feature column "Path" includes the image paths to be classified.
FEATURES = "Path"

#the same five labels as in the original CheXpert paper are considered: Atelectasis, Cardiomegaly, Consolidation, Edema and Pleural Effusion
LABELS = ["Atelectasis", "Cardiomegaly", "Consolidation", "Edema", "Pleural Effusion"]

#handling of the uncertainty labels: uncertain labels are mapped to one for the finding atelectasis and to zero for the other four diagnoses
U_ONES_LABELS = ["Atelectasis"]
U_ZEROS_LABELS = ["Cardiomegaly", "Consolidation", "Edema", "Pleural Effusion"]

#used metrics for performance evaluation and comparison: accuracy, AUROC, precision and recall (as F1-score)
METRICS = [BinaryAccuracy(name = "accuracy"), AUC(name = "auc", multi_label = True), Precision(name = "precision"), Recall(name = "recall")]


<b>FUNCTIONS</b></br>
Defining basic functions for preprocessing, reading in the images and plotting results, etc.

In [ ]:
#reads in the file names of the images and converts them into same-size RGB-images with padding, thereby keeping the aspect ratio
def parse_image(features, label):

    image_string = tf.io.read_file(features)
    image = tf.image.decode_jpeg(image_string, channels = CHANNELS)
    image = tf.image.resize_with_pad(image, IMAGE_SIZE, IMAGE_SIZE) #padding keeps original aspect ratio of the image
    image = tf.keras.applications.densenet.preprocess_input(image) #special preprocessing operation for the densenet structure

    return image, label

#creates a tensorflow dataset out of a pandas dataframe with the radiographs and selected labels
def create_dataset(dataframe):
    dataset = tf.data.Dataset.from_tensor_slices((dataframe[FEATURES].values, dataframe[LABELS].values))
    dataset = dataset.map(parse_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)

    return dataset

#preprocessing of the created dataset and image augmentation for the training data
def preprocess_dataset(dataset, is_training):
    dataset = dataset.cache().shuffle(int(len(dataset)/100), reshuffle_each_iteration = False)

    #augmentation for the training data, if IMAGE_AUGMENTATION is set to True
    if is_training == True and IMAGE_AUGMENTATION == True:
        print("Images in training dataset before augmentation: " + str(len(dataset)))
        dataset_augmented = dataset.take(int(AUGMENTATION_SPLIT*IMAGES*TRAIN_VALIDATION_SPLIT)).map(augment, num_parallel_calls = tf.data.experimental.AUTOTUNE)
        dataset = dataset.concatenate(dataset_augmented)
        print("Images in training dataset after augmentation: " + str(len(dataset)))

    dataset = dataset.batch(BATCH_SIZE).prefetch(buffer_size = tf.data.AUTOTUNE) #tensorflow input pipeline: batching the dataset and prefetching for increased efficiency

    return dataset

#image augmentation
def augment(image, label):
    image = tfa.image.rotate(image, random.uniform(-10, 10)*math.pi/180) #rotation of the image by up to 10 degrees in both directions
    image = tf.image.central_crop(image, central_fraction = random.uniform(0.8, 1.0)) #randomly zooming into the image by up to 20 percent
    image = tf.image.random_brightness(image, max_delta = 0.1) #manipulating the brightness by up to 10 percent
    image = tf.image.random_contrast(image, lower = 0.9, upper = 1.1) #manipulating the contrast by up to 10 percent
    image = tf.image.resize(image, [IMAGE_SIZE, IMAGE_SIZE]) #resizing the image due to previous central crop function

    return image, label

#plotting the loss and metric curves over the epochs
def plot_training(history):
    history_dict = history.history
    history_dict = list(history_dict)[:int(len(history_dict)/2)]

    #creating a pyplot with two columns and a fixed size
    num_rows = math.ceil(len(history_dict)/2)
    num_cols = 2
    pos = 1
    plt.figure(figsize = (13, 5*num_rows))

    #plotting training and validation curves for each metric and loss in one individual diagram
    for h in history_dict:
        plt.subplot(num_rows, num_cols, pos)
        plt.plot(history.history[h])
        plt.plot(history.history["val_" + h])
        plt.ylim([0.3, 0.9]) #fixing the range of the y-axis
        plt.title("model " + h, fontweight = "bold", fontsize = 13)
        plt.ylabel(h)
        plt.xlabel("epochs")
        plt.legend(["train", "valid"], loc = "best")
        pos += 1

#plotting a roc curve of the model for a selected dataset (training, validation or test)
def plot_roc_curve(data):
    if data == "training":
        dataset = train_ds
        pred = pred_train
        training_str = "training"
        pos = 1
    elif data == "validation":
        dataset = valid_ds
        pred = pred_valid
        training_str = "validation"
        pos = 2
    elif data == "test":
        dataset = test_ds
        pred = pred_test
        training_str = "test"
        pos = 3

    #getting the labels of the dataset
    b = np.concatenate([b for a, b in dataset], axis = 0)

    #initialising the value of the AUC sum
    auc_sum = 0.0

    #calculating the true- and false-positive rate of the model predictions for every diagnosis and thereby the AUROC metric
    for l in range(len(LABELS)):
        fpr, tpr, thresholds = roc_curve(b[:,l], pred[:,l])
        auc_metric = auc(fpr, tpr)
        plt.plot(fpr, tpr, label = LABELS[l] + " (AUC: " + str(round(auc_metric, 4)) + ")") #printing the indvidual metric values in the diagram
        auc_sum += auc_metric

    #creating a pyplot with a black bisector line for AUC = 0.5
    plt.plot([0, 1], [0, 1], 'k--')

    #defining title and axis descriptions
    plt.title("model ROC curve (" + training_str + ")", fontweight = "bold", fontsize = 13)
    plt.ylabel("True positive rate")
    plt.xlabel("False positive rate")
    plt.legend(loc = "best")

    #printing the average AUC value across all diagnoses
    auc_average = auc_sum/len(LABELS)
    print("Average AUC " + "(" + training_str + "): " + str(auc_average))

#plotting roc-curves for training, validation and test dataset
def plot_roc_curves():

    #creating a pyplot with two rows and columns
    plt.figure(figsize = (13, 10))
    num_rows = 2
    num_cols = 2

    plt.subplot(num_rows, num_cols, 1)
    plot_roc_curve("training") #plot training roc curve

    plt.subplot(num_rows, num_cols, 2)
    plot_roc_curve("validation") #plot validation roc curve

    plt.subplot(num_rows, num_cols, 3)
    plot_roc_curve("test") #plot test roc curve

#plotting a number of examplary images with their respective labels and neuronal network's predictions
def show_examples(data, number):
    number = min(number, 10)

    #selecting the right dataset
    if data == "training":
        dataset = train_ds
        pred = pred_train
    elif data == "validation":
        dataset = valid_ds
        pred = pred_valid
    elif data == "test":
        dataset = test_ds
        pred = pred_test

    num_rows = 1
    num_cols = 1
    pos = 1
    label_pred_str = ""

    for num in range(number): #loop for each radiograph

        index = random.randint(0, len(dataset)) #randomly picking rdiograph
        a, b = list(dataset.unbatch())[index] #getting the radiograph with its labels

        print("\033[1m" + "Image " + str(num + 1) + ":\t" + "\033[0m") #printing image umber

        for l in range(len(LABELS)): #printing the radiograph with its respective labels and predictions

            if(b[l] == 1.0):

                label_pred_str += str(LABELS[l] + ": ")
                label_pred_str += str(round(pred[index][l]*100, 2)) + " "

            print(LABELS[l])
            print(str(b.numpy()[l]) + "\t(Prediction: " + str(round(pred[index][l]*100, 2)) + "%)")

        fig = plt.figure(figsize = (13, 7)) #creating pyplot

        plt.subplot(num_rows, num_cols, pos)
        plt.imshow((a.numpy()*255).astype("uint8")) #printing radiograph
        plt.grid(None)
        plt.title("Image " + str(num + 1), fontweight = "bold", fontsize = 13)
        plt.xlabel(label_pred_str) #image subscription

        plt.show()
        label_pred_str = ""

<b>TRAIN & VALIDATION DATA FRAME</b></br>
Reading in the CheXpert dataset. It can be downloaded at: https://stanfordmlgroup.github.io/competitions/chexpert/
Subsequently filtering by frontal images and grouping by patient ID.

In [ ]:
dataframe = pd.read_csv("Chexpert/train.csv") #reading in the dataframe via the csv-file

#setting N/A labels to zero and uncertainty labels specific to u_ones and u_zeros in the CheXpert paper
for l in LABELS:
    if (l in U_ONES_LABELS):
        dataframe[l][dataframe[l] < 0] = 1
        dataframe[l] = dataframe[l].fillna(0)
    elif (l in U_ZEROS_LABELS):
        dataframe[l][dataframe[l] < 0] = 0
        dataframe[l] = dataframe[l].fillna(0)

#filtering out lateral radiographs
dataframe = dataframe[dataframe["Frontal/Lateral"] != "Lateral"][:min(191027, IMAGES)]

#creating a patient and study column
dataframe["Patient"] = dataframe.Path.str.split('/', n=3, expand=True)[2].str.split("patient", n=2, expand=True)[1]
dataframe["Patient"] = [i.lstrip("0") for i in dataframe["Patient"]]

dataframe["Study"] = dataframe.Path.str.split('/', n=4, expand=True)[3].str.split("study", n=2, expand=True)[1]

#grouping and shuffling the dataframe by patient
patients = dataframe["Patient"].unique()
random.shuffle(patients)
dataframe = dataframe.set_index("Patient").loc[patients]

dataframe.head()

In [ ]:
dataframe.shape 

<b>SELECTING CLIENT DATA</b></br>
Sorting and selecting the data depending on the federated learning scenario

In [ ]:
#Sorting the dataframe by age
dataframe = dataframe.sort_values(by='Age')  

# Calculating the indices for splitting the dataframe into five segments
segment_size = len(dataframe) // clients
start_index = (client_num - 1) * segment_size
end_index = start_index + segment_size

# include any remaining data points in last client
if client_num == clients:
    end_index = len(dataframe)

# Selecting the client's segment of the dataframe
dataframe = dataframe.iloc[start_index:end_index]

dataframe.shape  

<b>TEST DATA FRAME</b></br>
Reading in the test dataframe of the CheXpert dataset.

In [ ]:
dataframe_test = pd.read_csv("Chexpert/valid.csv") #reading in the test dataframe from the csv test file

dataframe_test = dataframe_test[dataframe_test["Frontal/Lateral"] != "Lateral"] #filtering out the lateral radiographs

dataframe_test.head()

In [ ]:
dataframe_test.shape #dimensions of the test dataframe:

<b>CREATE TRAIN & VALIDATION DATASET</b></br>
Creating a tf dataset

In [ ]:
dataset = create_dataset(dataframe) #creating tf dataset out of the pandas dataframe

<b>SPLIT TRAIN & VALIDATION DATASET</b></br>
Splitting the dataset into training and validation

In [ ]:
train_ds = dataset.take(int(TRAIN_VALIDATION_SPLIT*len(dataset))) #taking the training part of the dataset
valid_ds = dataset.skip(int(TRAIN_VALIDATION_SPLIT*len(dataset))) #taking the validation part of the dataset

train_ds = preprocess_dataset(train_ds, True) #preprocessing with augmentation
valid_ds = preprocess_dataset(valid_ds, False) #preprocessing without augmentation

<b>CREATE TEST DATASET</b></br>
Creating the test dataset

In [ ]:
test_ds = create_dataset(dataframe_test) #creating the test dataset
test_ds = preprocess_dataset(test_ds, False) #preprocessing without augmentation

<b>MODEL</b></br>Creating CNN structure for image classification

In [ ]:
#creating the base model
base_model = DenseNet121(
    include_top = False, #no default final classification layer
    weights = "imagenet", #Transfer learning with pretrained weights
    input_shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS),
    pooling = None, #no pooling
)

base_model.trainable = True #allow base model weight training

inputs = tf.keras.Input(shape = (IMAGE_SIZE, IMAGE_SIZE, CHANNELS)) #model input shape

#adding the different layers
x = base_model(inputs, training = True) #base model
x = tf.keras.layers.Conv2D(64, (3, 3))(x) #further convolutional layer
x = tf.keras.layers.GlobalAveragePooling2D()(x) #pooling layer
x = tf.keras.layers.BatchNormalization()(x) #batchNorm layer
x = tf.keras.layers.Dropout(0.4)(x) #dropout layer

#final dense layer for classification
outputs = tf.keras.layers.Dense(
    len(LABELS), #number of nodes equals the number of classes
    kernel_initializer = INITIALIZER, #initialising model weights with specific distribution
    activation = ACTIVATION #setting activation function
)(x)

model = tf.keras.Model(inputs, outputs)

#compiling the model
model.compile(
    loss = "binary_crossentropy", #model loss definition
    optimizer = Adam(learning_rate = LEARNING_RATE), #setting optimizer to Adam with fixed learning rate
    metrics = [BinaryAccuracy(name = "accuracy"), AUC(name = "auc", multi_label = True), Precision(name = "precision"), Recall(name = "recall")] #selecting predifined metrics
)

model.summary(expand_nested = False) #printing model summary

<b>TRAINING</b></br>
Defining FL client and training the model

In [ ]:
class Client(fl.client.NumPyClient):
    def get_parameters(self, **kwargs):  # Accept and ignore any additional keyword arguments
        return model.get_weights()

    def fit(self, parameters, config):
        model.set_weights(parameters)
        history = model.fit(train_ds, epochs=EPOCHS, validation_data=valid_ds)
        return model.get_weights(), len(train_ds), {}

    def evaluate(self, parameters, config):
        model.set_weights(parameters)
        loss, accuracy, auc, precision, recall = model.evaluate(train_ds)
        return loss, len(train_ds), {}


fl.client.start_numpy_client(server_address="localhost:8080", client=Client())


<b>MODEL EVALUATION ON TEST DATASET</b>

In [ ]:
model.evaluate(test_ds) #evaluating model performance on the test dataset

<b>MODEL PREDICTIONS</b></br>
Generating model predictions

In [ ]:
pred_train = model.predict(train_ds)
pred_valid = model.predict(valid_ds)
pred_test = model.predict(test_ds)

In [ ]:
plot_roc_curves()

<b>LIME EXPLANATIONS</b></br>
Generating LIME explanations for a subset of test images

In [ ]:

# Function to preprocess the image for LIME
def preprocess_image_for_lime(path_to_image, img_size):
    image_string = tf.io.read_file(path_to_image)
    image = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    image = tf.image.resize_with_pad(image, img_size, img_size)
    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, 0)
    image = tf.keras.applications.densenet.preprocess_input(image)
    return image

# Update the batch_predict function to process a batch of tensor images
def batch_predict(images):
    batch_images = tf.concat([preprocess_image_for_lime(path, IMAGE_SIZE) for path in images], axis=0)
    preds = model.predict(batch_images)
    return preds

   
# Creating the LIME explainer
explainer = lime_image.LimeImageExplainer()

# Defining the segmentation algorithm
segmenter = SegmentationAlgorithm('quickshift', kernel_size=4, max_dist=200, ratio=0.2)

# Paths to images used for the explanations
specific_paths_to_images = [
    'Chexpert/valid/patient64639/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64719/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64611/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64591/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64599/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64572/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64564/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64672/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64655/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64704/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64585/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64623/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64598/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64715/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64650/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64646/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64632/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64588/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64651/study1/view1_frontal.jpg',
    'Chexpert/valid/patient64604/study1/view1_frontal.jpg'
]

# Store the explanation masks for consistency assessment
explanation_masks = []

# Iterate over selected images
for path_to_image in specific_paths_to_images:
    # Output the path of the image
    print(f"Processing Image: {path_to_image}")

    image_tensor = preprocess_image_for_lime(path_to_image, IMAGE_SIZE)
    image_for_display = np.squeeze(image_tensor.numpy())

    # Display the original image
    plt.imshow(image_for_display / 2 + 0.5)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()
    
    # Generate explanation for the image
    explanation = explainer.explain_instance(image_for_display.astype('double'), 
                                             model.predict,  
                                             top_labels=5, 
                                             hide_color=0, 
                                             num_samples=100, 
                                             segmentation_fn=segmenter)
    
    top_label_index = explanation.top_labels[0]
    top_label_name = LABELS[top_label_index]

    _, mask = explanation.get_image_and_mask(top_label_index, positive_only=False, num_features=10, hide_rest=False)
    
    # Append the generated mask to the explanation_masks list
    explanation_masks.append(mask)

    # Display the explanation on the image
    plt.imshow(mark_boundaries(image_for_display / 2 + 0.5, mask))
    plt.title(f'Explanation for label: {top_label_name}')
    plt.axis('off')
    plt.show()

In [ ]:
<b>Saving metrics</b></br>
Saving metrics of explanations for later model comparisons

In [ ]:
# Directory to save the explanations
explanation_dir = "workspace/explanations_fl_5_5_FedAvg_001"
os.makedirs(explanation_dir, exist_ok=True)

# Save the masks
for i, mask in enumerate(explanation_masks):
    # Save the mask to a compressed .npz file
    mask_path = os.path.join(explanation_dir, f"explanation_mask_fl_5_5_FedAvg_001_{i}.npz")
    np.savez_compressed(mask_path, mask=mask)

# Save the manifest with details about the images and models
manifest_path = os.path.join(explanation_dir, "explanation_manifest_fl_5_5_FedAvg_001.csv")
with open(manifest_path, 'w') as file:
    file.write("explanation_id,image_path,model_details\n")
    for i, path_to_image in enumerate(specific_paths_to_images):
        file.write(f"{i},{path_to_image},central_model_001\n")  


<b>Calculating Local Fidelity Scores</b></br>

In [ ]:
# Create an empty list to store Local Fidelity Scores
local_fidelity_scores = []

# Iterate over selected images
for path_to_image in specific_paths_to_images:
    # Output the path of the image
    print(f"Processing Image: {path_to_image}")

    image_tensor = preprocess_image_for_lime(path_to_image, IMAGE_SIZE)
    image_for_display = np.squeeze(image_tensor.numpy())

    # Display the original image
    plt.imshow(image_for_display / 2 + 0.5)
    plt.title('Original Image')
    plt.axis('off')
    plt.show()

    explanation = explainer.explain_instance(image_for_display.astype('double'), 
                                             model.predict,  
                                             top_labels=5, 
                                             hide_color=0, 
                                             num_samples=1000, 
                                             segmentation_fn=segmenter)
    
    top_label_index = explanation.top_labels[0]
    top_label_name = LABELS[top_label_index]

    # Get the explanation mask
    _, mask = explanation.get_image_and_mask(top_label_index, positive_only=False, num_features=10, hide_rest=False)

    # Ensure the explanation mask and image tensor have the same shape
    mask = tf.image.resize(tf.expand_dims(mask, -1), (IMAGE_SIZE, IMAGE_SIZE))  
    mask = tf.cast(mask, tf.float32)

    # Perform element-wise multiplication between the mask and image_tensor
    masked_image = mask * image_tensor

    # Apply the same preprocessing as used for model training data
    masked_image = tf.keras.applications.densenet.preprocess_input(masked_image)

    # Get the model's prediction for the masked image
    prediction_with_explanation = model.predict(masked_image)

    # Compare the model's prediction with the LIME's prediction
    local_fidelity_score = np.mean(np.abs(model.predict(image_tensor) - prediction_with_explanation))
    print(f"Local Fidelity Score for {path_to_image}: {local_fidelity_score}")

    # Append the Local Fidelity Score to the list
    local_fidelity_scores.append(local_fidelity_score)

    # Convert the mask to an integer data type for processing boundaries
    mask_as_int = (mask * 255).numpy().astype(np.uint8)

    # Display the explanation on the image
    plt.imshow(mark_boundaries(image_for_display / 2 + 0.5, mask_as_int[:, :, 0]))
    plt.title(f'Explanation for label: {top_label_name}')
    plt.axis('off')
    plt.show()

# Save the Local Fidelity Scores to a CSV file
import csv
folder_path = "workspace/explanations_fl_5_5_FedAvg_001/"
if not os.path.exists(folder_path):
    os.makedirs(folder_path)
csv_file = os.path.join(folder_path, "local_fidelity_scores_fl_5_5_FedAvg_001.csv")
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_Path", "Local_Fidelity_Score"]) 
    
    # Iterate over selected images and write to CSV
    for i, path_to_image in enumerate(specific_paths_to_images):
        writer.writerow([path_to_image, local_fidelity_scores[i]])
        print(f"Writing to CSV: {path_to_image}, {local_fidelity_scores[i]}")


    # Iterate over selected images
    for path_to_image in specific_paths_to_images:
        # Calculate the local fidelity score
        local_fidelity_score = np.mean(np.abs(model.predict(image_tensor) - prediction_with_explanation))

        # Print the local fidelity score
        print(f"Local Fidelity Score for {path_to_image}: {local_fidelity_score}")

        # Append the local fidelity score to the list
        local_fidelity_scores.append(local_fidelity_score)

# Save the local fidelity scores to the CSV file
with open(csv_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    for i, path_to_image in enumerate(specific_paths_to_images):
        writer.writerow([path_to_image, local_fidelity_scores[i]])

